In [ ]:
#1)
dftrain=read.csv(file.choose())
dftrain=dftrain[sample(nrow(dftrain),25000),]
dftrain=dftrain %>% drop_na()
dftest=read.csv(file.choose())


install.packages('fastDummies')
library('fastDummies')
library(caret)
library(class)
library(e1071)
library(tidyverse)
library(knitr)
library('stats')

#training CSV

#features of interest: wave index, courier_wave_start_lng, courier_wave_start_lat, date, group, action type, level, speed, max load, weather grade, source_type, source_tracking_id,source_lng,source_lat, target_lng, target_lat,grid_distance, urgency,hour
#label: expected_use_time

dftrain$action_type=as.numeric(dftrain$action_type)
dftrain$weather_grade=as.numeric(dftrain$weather_grade)
dftrain$source_type=as.numeric(dftrain$source_type)

#dummy var
dftrain$weather_grade=ifelse(dftrain$weather_grade=='Bad Weather',1,0)
dftrain$source_type=ifelse(dftrain$source_type=='Delivery',1,0)
dftrain$action_type=ifelse(dftrain$source_type=='Delivery',1,0)


#remove cols
dftrain$courier_id=NULL
dftrain$tracking_id=NULL
dftrain$aoi_id=NULL
dftrain$shop_id=NULL
dftrain$id=NULL


#cut down data size
set.seed(77)
training.rows=sample(1:nrow(dftrain),nrow(dftrain)*0.7)
df_train=dftrain[training.rows,]
df_test=dftrain[-training.rows,]

#linear regression model

#model w/ all variables except one's removed in NULL above
regmodel1=lm(expected_use_time~wave_index+courier_wave_start_lng+courier_wave_start_lat+date+action_type+level+speed+max_load+weather_grade+source_type+source_tracking_id+source_lng+source_lat+target_lng+target_lat+grid_distance+urgency+hour,data=df_train)
regmodel2=lm(expected_use_time~date+action_type+level+speed+max_load+weather_grade+source_type+grid_distance+urgency+hour,data=df_train)

time=predict(regmodel1,newdata=df_test)
SSE=sum((time-df_test$expected_use_time)^2)

SST=sum((df_test$expected_use_time-mean(df_train$expected_use_time))^2)
cat('The out-of-sample R-squared for the regmodel1 is',1-SSE/SST)

regmodel2=lm(expected_use_time~date+action_type+level+speed+max_load+weather_grade+source_type+grid_distance+urgency+hour,data=df_train)

time1=predict(regmodel2,newdata=df_test)
SSE1=sum((time1-df_test$expected_use_time)^2)

SST1=sum((df_test$expected_use_time-mean(df_train$expected_use_time))^2)
cat('The out-of-sample R-squared for the regmodel1 is',1-SSE1/SST1)

#MAE calc
library(Metrics)

#regmodel1
MAE1=mae(df_test$expected_use_time,time)

#regmodel2
MAE2=mae(df_test$expected_use_time,time1)

#standardization

istime=df_train$expected_use_time
istimet=df_test$expected_use_time

#train
x1=(df_train$wave_index-mean(df_train$wave_index))/sd(df_train$wave_index)
x2=(df_train$courier_wave_start_lng-mean(df_train$courier_wave_start_lng))/sd(df_train$courier_wave_start_lng)
x3=(df_train$courier_wave_start_lat-mean(df_train$courier_wave_start_lat))/sd(df_train$courier_wave_start_lat)
x4=(df_train$action_type-mean(df_train$action_type))/sd(df_train$action_type)
x5=(df_train$date-mean(df_train$date))/sd(df_train$date)
x6=(df_train$group-mean(df_train$group))/sd(df_train$group)
x7=(df_train$level-mean(df_train$level))/sd(df_train$level)
x8=(df_train$speed-mean(df_train$speed))/sd(df_train$speed)
x9=(df_train$max_load-mean(df_train$max_load))/sd(df_train$max_load)
x10=(df_train$weather_grade-mean(df_train$weather_grade))/sd(df_train$weather_grade)
x11=(df_train$source_type-mean(df_train$source_type))/sd(df_train$source_type)
x12=(df_train$source_tracking_id-mean(df_train$source_tracking_id))/sd(df_train$source_tracking_id)
x13=(df_train$source_lng-mean(df_train$source_lng))/sd(df_train$source_lng)
x14=(df_train$source_lat-mean(df_train$source_lat))/sd(df_train$source_lat)
x15=(df_train$target_lat-mean(df_train$target_lat))/sd(df_train$target_lat)
x16=(df_train$target_lng-mean(df_train$target_lng))/sd(df_train$target_lng)
x17=(df_train$grid_distance-mean(df_train$grid_distance))/sd(df_train$grid_distance)
x18=(df_train$urgency-mean(df_train$urgency))/sd(df_train$urgency)
x19=(df_train$hour-mean(df_train$hour))/sd(df_train$hour)
r=as.factor(df_train$expected_use_time)

knn_train=data.frame(x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19)
knn_training=data.frame(x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,r)

#test
v1=(df_test$wave_index-mean(df_test$wave_index))/sd(df_test$wave_index)
v2=(df_test$courier_wave_start_lng-mean(df_test$courier_wave_start_lng))/sd(df_test$courier_wave_start_lng)
v3=(df_test$courier_wave_start_lat-mean(df_test$courier_wave_start_lat))/sd(df_test$courier_wave_start_lat)
v4=(df_test$action_type-mean(df_test$action_type))/sd(df_test$action_type)
v5=(df_test$date-mean(df_test$date))/sd(df_test$date)
v6=(df_test$group-mean(df_test$group))/sd(df_test$group)
v7=(df_test$level-mean(df_test$level))/sd(df_test$level)
v8=(df_test$speed-mean(df_test$speed))/sd(df_test$speed)
v9=(df_test$max_load-mean(df_test$max_load))/sd(df_test$max_load)
v10=(df_test$weather_grade-mean(df_test$weather_grade))/sd(df_test$weather_grade)
v11=(df_test$source_type-mean(df_test$source_type))/sd(df_test$source_type)
v12=(df_test$source_tracking_id-mean(df_test$source_tracking_id))/sd(df_test$source_tracking_id)
v13=(df_test$source_lng-mean(df_test$source_lng))/sd(df_test$source_lng)
v14=(df_test$source_lat-mean(df_test$source_lat))/sd(df_test$source_lat)
v15=(df_test$target_lat-mean(df_test$target_lat))/sd(df_test$target_lat)
v16=(df_test$target_lng-mean(df_test$target_lng))/sd(df_test$target_lng)
v17=(df_test$grid_distance-mean(df_test$grid_distance))/sd(df_test$grid_distance)
v18=(df_test$urgency-mean(df_test$urgency))/sd(df_test$urgency)
v19=(df_test$hour-mean(df_test$hour))/sd(df_test$hour)
r=as.factor(df_test$expected_use_time)

knn_test=data.frame(v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,v18,v19)
knn_testing=data.frame(v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,v18,v19,r)




#reg tree
library('rpart')
library('rpart.plot')
timetree=rpart(expected_use_time~.,data=df_train, method="class",parms=list(split='gini'),cp=0.001)
prp(timetree,extra=1)

#prediction
predictValid=predict(timetree,newdata=df_test,type="prob")[,18]

#AUC
library(AUC)
rocv=roc(predictValid,as.factor(df_test$expected_use_time))
aucv=auc(rocv)
print(paste('The out-of-sample AUC for the decision tree is', aucv))
plot(rocv)

MAE1=mae(df_test$expected_use_time,predictValid)


#Feature Engineering - clustering
k=3
df_train=unique(df_train)
clusterdf=kmeans(df_train,centers=5, iter.max=1000)
nrow(subset(df_train,clusterdf$cluster==1))
nrow(subset(df_train,clusterdf$cluster==2))
nrow(subset(df_train,clusterdf$cluster==3))

#remove features (geography only)
df_train$wave_index=NULL
df_train$action_type=NULL
df_train$date=NULL
df_train$group=NULL
df_train$level=NULL
df_train$speed=NULL
df_train$max_load=NULL
df_train$weather_grade=NULL
dftrain$source_type=NULL
df_train$urgency=NULL
df_train$hour=NULL

df_train$expected_use_time=as.factor(df_train$expected_use_time)



library(flexclust)
time.kcca=as.kcca(clusterdf,df_train)
timecluster=predict(time.kcca,newdata=df_test)

nrow(subset(df_test,timecluster==1))
nrow(subset(df_test,timecluster==2))
nrow(subset(df_test,timecluster==3))

timetrain=subset(df_train,clusterdf$cluster==1)
timetrain1=subset(df_train,clusterdf$cluster==2)
timetrain2=subset(df_train,clusterdf$cluster==3)
timetest=subset(df_test,timecluster==1)
timetest1=subset(df_test,timecluster==2)
timetest2=subset(df_test,timecluster==3)

timelog=glm(expected_use_time~.,data=timetrain,family=binomial)
timelog1=glm(expected_use_time~.,data=timetrain1,family=binomial)
timelog2=glm(expected_use_time~.,data=timetrain2,family=binomial)

predicttest1=predict(timelog,type="response",newdata=timetest)
predicttest2=predict(timelog1,type="response",newdata=timetest1)
predicttest3=predict(timelog2,type="response",newdata=timetest2)

#cluster1 AUC
predtest.n=as.numeric(predicttest1)
df_test.f=as.factor(df_test$expected_use_time)
roct1=roc(predtest.n,df_test.f)
auct1=auc(roct1)
print(paste('The out_of_sample AUC with clustering and prediction is',auct1))

MAEc1=mae(df_test$expected_use_time,predtest.n)


#cluster2 AUC
predtest1.n=as.numeric(predicttest2)
roct2=roc(predtest1.n,df_test.f)
auct2=auc(roct2)
print(paste('The out_of_sample AUC with clustering and prediction is',auct2))

MAEc2=mae(df_test$expected_use_time,predtest1.n)

#cluster3 AUC
predtest2.n=as.numeric(predicttest3)
roct3=roc(predtest2.n,df_test.f)
auct3=auc(roct3)
print(paste('The out_of_sample AUC with clustering and prediction is',auct3))

MAEc3=mae(df_test$expected_use_time,predtest2.n)

#2)
install.packages('haven')
library('haven')
llibrary('tidyverse')

df=as.data.frame(read_dta(file.choose()))
df1=as.data.frame(read_dta(file.choose()))
df2=as.data.frame(read_dta(file.choose()))

df=df %>% drop_na()
df1=df1 %>% drop_na()
df2=df2 %>% drop_na()

x=merge(df,df1,by.x="person_id",by.y="person_id",all.x=T,all.y=F)
x=merge(total,df2,by.x='person_id',b.y="person_id",all.x=F,all.y=T)

#dummy var
x$numhh_list=ifelse(x$numhh_list=='1',1,0)

#balance check
t.test(x$treatment,x$any_on_ed)
b1=lm(treatment~any_on_ed,data=x)
t.test(x$treatment,x$birthyear_list)
b2=lm(treatment~birthyear_list,data=x)
t.test(x$treatment,x$female_list)
b3=lm(treatment~female_list,data=x)
t.test(x$treatment,x$self_list)
b4=lm(treatment~self_list,data=x)
t.test(x$treatment,x$any_visit_pre_ed)
b5=lm(treatment~any_visit_pre_ed,data=x)
t.test(x$treatment,x$num_visit_pre_cens_ed)
b6=lm(treatment~num_visit_pre_cens_ed,data=x)

t.test(x$numhh_list,x$any_on_ed)
b7=lm(numhh_list~any_on_ed,data=x)
t.test(x$numhh_list,x$birthyear_list)
b8=lm(numhh_list~birthyear_list,data=x)
t.test(x$numhh_list,x$female_list)
b9=lm(numhh_list~female_list,data=x)
t.test(x$numhh_list,x$self_list)
b10=lm(numhh_list~self_list,data=x)
t.test(x$numhh_list,x$any_visit_pre_ed)
b11=lm(numhh_list~any_visit_pre_ed,data=x)
t.test(x$numhh_list,x$num_visit_pre_cens_ed)
b12=lm(numhh_list~num_visit_pre_cens_ed,data=x)

#causal effect of being selected

linearm=lm(ohp_all_mo_firstn_30sep2009~any_on_ed+birthyear_list+female_list+self_list+any_visit_pre_ed+num_visit_pre_cens_ed,data=x)
summary(linearm)

ATE=mean(treatment$ohp_all_mo_firstn_30sep2009)-mean(numhh_list$ohp_all_mo_firstn_30sep2009)

linearm1=lm(ohp_all_mo_firstn_30sep2009~any_on_ed+birthyear_list+female_list+self_list+any_visit_pre_ed+num_visit_pre_cens_ed+draw_treat+approved_app+first_day_list,data=x)


install.packages('Matching',repos = "https://mirrors.tuna.tsinghua.edu.cn/CRAN/")
library(MASS)
library("Matching")
pscorem=glm(ohp_all_end_30sep2009~treatment,data=x,family=binomial)
pscore=predict(pscorem,type='response')
x$pscore=pscore
head(x)

